# **컴퓨터에 금융 시장 데이터를 저장하기**

알고리즘 트레이더들이 항상 부족함을 느끼는 한 가지가 있다면, 그것은 데이터입니다. 전략의 기반이 되는 데이터는 단순한 숫자를 넘어 우리의 의사결정 과정을 지탱하는 핵심 요소입니다. 데이터를 로컬 또는 적어도 여러분의 통제 하에 보유하는 것은 매우 중요합니다. 접근 속도와 신뢰성이 데이터를 로컬에 저장해야 하는 주요 이유입니다. 로컬 데이터는 인터넷 중단으로부터 보호되며, 데이터 기반 프로세스가 중단 없이 작동하도록 보장합니다. 또한, 잘못된 가격을 수정해야 할 경우에도 그 수정 사항을 지속적으로 유지할 수 있습니다.

비용 측면에서 보면, 로컬 저장은 반복적으로 발생하는 클라우드 비용에 비해 비용 효율적인 이점이 있습니다. 클라우드 기반 데이터베이스에 몇 테라바이트의 데이터를 저장하는 것은 매달 수백 달러가 들 수 있습니다. 데이터 조작의 유연성, 연구 워크플로와의 통합 용이성, 백테스트 속도의 향상은 추가적인 장점입니다.

이 장에서는 금융 시장 데이터를 저장하는 여러 방법을 탐구합니다. 먼저 CSV 파일에 데이터를 저장하는 것부터 시작하며, 이는 Pandas로 쉽게 읽고 쓸 수 있습니다. 이후 간단한 온디스크 SQL 데이터베이스 형식인 SQLite를 사용해 데이터를 저장하는 방법을 탐구합니다. 복잡성을 증가시켜 PostgreSQL 데이터베이스 서버를 컴퓨터에 설치해 데이터를 저장합니다. 마지막으로 초고속 HDF5 형식을 사용해 데이터를 저장하는 방법을 다룹니다.

SQLite와 PostgreSQL을 사용하는 예제에서는, 시장 종료 후 데이터를 자동으로 가져올 수 있도록 작업 관리자를 사용해 실행할 수 있는 스크립트를 개발합니다.

이 장에서 다룰 레시피는 다음과 같습니다:

- CSV 형식으로 디스크에 데이터 저장하기
- SQLite를 사용해 디스크에 데이터 저장하기
- 네트워크 PostgreSQL 데이터베이스에 데이터 저장하기
- 초고속 HDF5 형식으로 데이터 저장하기

---

## **CSV 형식으로 디스크에 데이터 저장하기**

콤마로 구분된 값(CSV) 형식은 가장 널리 인식되고 활용되는 데이터 저장 방법 중 하나입니다. 그 단순성으로 인해 더 복잡한 시스템의 부하 없이 표 형식 데이터를 저장하려는 트레이더와 분석가들에게 선호되는 선택입니다. 알고리즘 트레이더는 주로 Python 및 그 라이브러리(예: Pandas)를 통해 CSV 파일을 처리하기가 쉬워 간단한 가져오기 및 내보내기 작업이 필요한 데이터를 처리할 때 CSV를 선호합니다. 더불어 CSV 형식의 데이터는 Tableau, PowerBI 또는 독점 시스템과 같은 다른 분석 도구와도 사용할 수 있습니다.

CSV 파일은 텍스트 편집기나 Excel을 사용하여 수동으로 검사할 수도 있습니다. CSV는 다른 저장 방법만큼 빠르거나 정교하지는 않지만, 사용의 용이성 덕분에 모든 거래 환경에서 중요한 역할을 합니다.

### **어떻게 할까요...**

Pandas는 데이터를 CSV로 저장하는 기능을 기본적으로 지원하므로, 특별한 라이브러리가 필요하지 않습니다.

1. **라이브러리 가져오기**:

In [2]:
import pandas as pd
from openbb import obb

In [3]:
obb.user.preferences.output_type = "dataframe"

2. **데이터를 다운로드하고 결과를 조작하며 pandas DataFrame을 반환하는 함수 구현**:

주어진 종목과 날짜 범위에 대한 주가 데이터를 가져오고 'symbol' 열을 추가하는 함수

In [4]:
def get_stock_data(symbol, start_date=None, end_date=None):
    """
    주어진 종목과 날짜 범위에 대한 주가 데이터를 가져오는 함수
    
    매개변수:
        symbol (str): 주식 종목 코드
        start_date (str, optional): 시작일. 기본값은 None
        end_date (str, optional): 종료일. 기본값은 None
        
    반환값:
        pandas.DataFrame: 주가 데이터가 포함된 데이터프레임
    """
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

3. **CSV 파일로 데이터를 저장하는 함수 구현**:

가져온 주식 데이터를 gzip으로 압축된 CSV 파일로 저장하는 함수

In [5]:
def save_data_range(symbol, start_date=None, end_date=None):
    """
    주어진 종목과 날짜 범위의 주가 데이터를 gzip으로 압축된 CSV 파일로 저장하는 함수
    
    매개변수:
        symbol (str): 주식 종목 코드
        start_date (str, optional): 시작일. 기본값은 None
        end_date (str, optional): 종료일. 기본값은 None
    """
    data = get_stock_data(symbol, start_date, end_date)
    data.to_csv(f"{symbol}.gz", compression="gzip", index=False)

4. **CSV 파일을 읽고 DataFrame을 반환하는 함수 구현**:

gzip으로 압축된 CSV 파일에서 주가 데이터를 읽는 함수

In [6]:
def get_data(symbol):
    """
    gzip으로 압축된 CSV 파일에서 주가 데이터를 읽어오는 함수
    
    매개변수:
        symbol (str): 주식 종목 코드
        
    반환값:
        pandas.DataFrame: 주가 데이터가 포함된 데이터프레임
    """
    return pd.read_csv(
        f"{symbol}.gz",
        compression="gzip", 
        index_col="date",
        usecols=["date", "open", "high", "low", "close", "volume", "symbol"],
    )

5. **데이터를 CSV 파일로 저장하기**:

주식 종목 "PLTR"의 데이터 저장하기

In [7]:
save_data_range("PLTR")

"PLTR"에 대해 저장된 데이터 가져오기

In [8]:
pltr = get_data("PLTR")

DataFrame 'df'를 CSV 파일로 저장하기

In [12]:
import os

if not os.path.exists("./datasets"):
    os.makedirs("./datasets")
    
pltr.to_csv("./datasets/market_data.csv")

DataFrame 'df'를 탭으로 구분된 파일로 저장하기

In [13]:
pltr.to_csv("./datasets/market_data.tsv", sep="\t")

'df'의 특정 열을 CSV 파일로 저장하기

In [14]:
pltr.to_csv("./datasets/market_data.csv", columns=["open", "close"])

특정 날짜 형식으로 'df'를 CSV 파일로 저장하기

In [15]:
pltr.to_csv("./datasets/market_data.csv", date_format="%Y-%m-%d")

CSV 파일의 처음 10행 읽기

In [16]:
pltr = pd.read_csv("./datasets/market_data.csv", nrows=10)

CSV 파일의 처음 10행을 건너뛰고 나머지 행 읽기

In [17]:
pltr = pd.read_csv("./datasets/market_data.csv", skiprows=range(1, 11))

CSV 파일을 읽고 'NULL' 값을 NaN으로 처리하도록 지정하기

In [18]:
pltr = pd.read_csv("./datasets/market_data.csv", na_values="NULL")

### **작동 방식...**

이 스크립트는 `pandas` 라이브러리와 OpenBB 플랫폼을 사용하여 주식 시장 데이터를 다운로드하고 조작합니다. `get_stock_data` 함수는 지정된 주식 심볼(symbol)과 날짜 범위를 사용하여 데이터를 가져오고, 인덱스를 재설정하고 열 이름을 표준화하여 데이터를 전처리합니다. 참조용으로 주식 심볼 열도 추가됩니다.

`save_data_range`는 `pandas`의 `to_csv` 메서드를 사용해 데이터를 CSV 형식으로 GZIP 압축하여 디스크에 저장합니다. 저장된 파일의 이름 규칙은 주식 심볼 뒤에 `.gz` 확장자가 붙는 형태입니다.

`get_data` 함수는 저장된 CSV 파일을 검색하고 압축을 해제하여 이를 DataFrame으로 재구성합니다. 이 과정에서 날짜, 시작 가격(open), 최고 가격(high), 최저 가격(low), 종료 가격(close), 거래량(volume), 주식 심볼(symbol)과 같은 선택된 열만 로드됩니다.

스크립트의 마지막 부분에서는 이러한 함수를 사용해 심볼 "PLTR"에 대한 주식 데이터를 저장합니다. 이 코드를 실행하면 컴퓨터에 `PLTR.gz`라는 이름의 압축된 CSV 파일이 생성됩니다.


---

### **추가 정보...**

`pandas`의 `to_csv` 메서드는 데이터를 CSV 형식으로 효율적으로 저장하기 위한 다양한 옵션을 제공합니다. 유용한 옵션은 다음과 같습니다:

- **`sep`**: 필드 간 구분자로 사용할 문자를 지정합니다. 기본값은 쉼표(,)이며, 탭으로 구분된 파일의 경우 `sep='\\t'`를 사용할 수 있습니다.
- **`header`**: 열 이름을 포함할지 여부를 결정하는 불리언 값입니다. `False`로 설정하면 CSV에서 열 헤더가 제외됩니다.
- **`na_rep`**: 결측값(NaN)의 문자열 표현을 설정합니다. 기본값은 빈 문자열이지만 `NULL`과 같은 플레이스홀더로 변경할 수 있습니다.
- **`date_format`**: 날짜 및 시간 개체의 형식을 지정합니다. 예: `date_format='%Y-%m-%d %H:%M:%S'`는 날짜를 `2023-08-10 15:20:30` 형태로 포맷합니다.
- **`float_format`**: 부동 소수점 숫자의 형식을 제어합니다. 예: `float_format='%.2f'`는 모든 부동 소수점 열을 소수점 둘째 자리로 반올림합니다.

마찬가지로, `get_data` 메서드에서 사용된 `read_csv` 메서드는 디스크에서 데이터를 가져오는 데 유연성을 추가하는 여러 옵션을 제공합니다:

- **`delimiter` 또는 `sep`**: 필드를 구분하는 문자를 지정합니다. 기본값은 쉼표(,)이며, 탭으로 구분된 파일의 경우 `delimiter='\\t'`를 사용할 수 있습니다.
- **`nrows`**: 읽을 파일의 행 수를 결정합니다. 대용량 파일의 일부분만 읽는 데 유용합니다.
- **`parse_dates`**: 날짜로 구문 분석할 열 이름 목록입니다. 예: `['date']`를 전달하면 `date` 열이 `datetime64` 형식으로 변환됩니다.
- **`dtype`**: 각 열에 사용할 열 이름 및 데이터 유형의 사전을 제공합니다. 예: `dtype={'volume': 'int32'}`는 `volume` 열을 32비트 정수로 읽도록 보장합니다.
- **`skiprows`**: 읽는 동안 생략할 행 번호의 목록이나 숫자를 지정합니다. 특정 행을 제외할 때 유용합니다.

---

### **참고 자료...**

시장 데이터를 다룰 때 CSV로 데이터를 읽고 쓰는 작업은 매우 일반적입니다. CSV와 관련된 다양한 방법을 익히는 것이 중요합니다:

- `to_csv` 메서드의 문서: [pandas.DataFrame.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)
- `read_csv` 메서드의 문서: [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

---



## **SQLite를 사용해 디스크에 데이터 저장하기**

SQLite는 평문 파일의 단순함과 관계형 데이터베이스의 강력함 사이의 다리를 제공합니다. 서버리스이면서 독립 실행형 데이터베이스인 SQLite는 알고리즘 트레이더들에게 SQL을 사용해 데이터를 저장하고 쿼리할 수 있는 가볍고도 강력한 도구를 제공합니다. 이는 완전한 데이터베이스 시스템을 구축할 필요 없이 가능합니다. Python과의 통합이 매끄럽고, 소형의 특성 덕분에 이동성과 최소한의 설정이 중요한 애플리케이션에 적합합니다. CSV보다 더 많은 구조를 필요로 하거나, SQL을 선호하지만 대규모 데이터베이스 시스템의 복잡성을 피하고 싶은 트레이더에게 SQLite는 최적의 선택입니다.

---

### **준비하기...**

이 예제에서는 CRON 작업(Mac, Linux, Unix)이나 작업 스케줄러(Windows)를 사용하여 자동으로 실행할 수 있는 스크립트를 작성합니다. 우리는 `market_data_sqlite.py`라는 Python 스크립트를 작성하고 이를 명령줄에서 실행할 것입니다. 또한 `exchange_calendars` Python 패키지를 소개합니다. 이 패키지는 50개 이상의 거래소에 대한 거래일과 시간을 정의하고 쿼리하는 캘린더를 제공합니다. 설치 명령은 다음과 같습니다:

In [ ]:
# !pip install exchange-calendars

### **어떻게 할까요...**

다음의 모든 코드는 `market_data_sqlite.py` 스크립트 파일에 작성해야 합니다:

1. **필요한 라이브러리 가져오기**:

In [19]:
import sqlite3

In [20]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb

In [21]:
obb.user.preferences.output_type = "dataframe"

2. **이전 레시피에서 `get_stock_data` 함수 재사용하기**:

주어진 심볼과 날짜 범위에 대한 주식 데이터를 가져오고 'symbol' 열을 추가하는 함수

In [22]:
def get_stock_data(symbol, start_date=None, end_date=None):
    """
    주어진 심볼과 날짜 범위에 대한 주식 데이터를 가져오는 함수
    
    Parameters:
        symbol (str): 주식 심볼 (예: 'AAPL', 'MSFT' 등)
        start_date (str, optional): 시작 날짜. 기본값은 None
        end_date (str, optional): 종료 날짜. 기본값은 None
        
    Returns:
        pandas.DataFrame: 주식 데이터가 포함된 데이터프레임
            - date: 날짜
            - open: 시가
            - high: 고가 
            - low: 저가
            - close: 종가
            - volume: 거래량
            - symbol: 주식 심볼
    """
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

3. **`save_data_range` 함수를 `pandas to_sql` 메서드로 수정하기**:

가져온 주식 데이터를 SQLite 데이터베이스에 저장하는 함수

In [23]:
def save_data_range(symbol, conn, start_date=None, end_date=None):
    """
    주어진 심볼과 날짜 범위의 주식 데이터를 SQLite 데이터베이스에 저장하는 함수
    
    Parameters:
        symbol (str): 주식 심볼 (예: 'AAPL', 'MSFT' 등)
        conn: SQLite 데이터베이스 연결 객체
        start_date (str, optional): 시작 날짜. 기본값은 None
        end_date (str, optional): 종료 날짜. 기본값은 None
    """
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", conn, if_exists="append", index=False)

4. **거래소의 캘린더를 기준으로 마지막 거래일 데이터를 가져오는 함수 생성하기**:

마지막 거래일의 주식 데이터를 SQLite 데이터베이스에 저장하는 함수

In [24]:
def save_last_trading_session(symbol, conn):
    """
    마지막 거래일의 주식 데이터를 SQLite 데이터베이스에 저장하는 함수
    
    Parameters:
        symbol (str): 주식 심볼 (예: 'AAPL', 'MSFT' 등)
        conn: SQLite 데이터베이스 연결 객체
    """
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", conn, if_exists="append", index=False)

SQLite 데이터베이스에 연결 설정하기

In [27]:
import os

if not os.path.exists("./datasets"):
    os.makedirs("./datasets")
    
conn = sqlite3.connect("./datasets/market_data.sqlite")

지정된 날짜 범위에서 여러 주식 심볼에 대한 데이터 저장하기

In [28]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(symbol, conn=conn, start_date="2020-06-01", end_date="2023-01-01")

데이터베이스에서 주식 심볼 "SPY"에 대한 데이터를 읽고 표시하기

In [29]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", conn)
display(df_1)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.0,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.0,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.0,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.0,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,SPY
...,...,...,...,...,...,...,...,...
648,2022-12-23,379.649994,383.059998,378.029999,382.910004,59857300,0.0,SPY
649,2022-12-27,382.790009,383.149994,379.649994,381.399994,51638200,0.0,SPY
650,2022-12-28,381.329987,383.390015,376.420013,376.660004,70911500,0.0,SPY
651,2022-12-29,379.630005,384.350006,379.079987,383.440002,66970900,0.0,SPY


"SPY" 심볼에 대해 거래량이 100,000,000 이상인 데이터를 읽고 표시하기

In [30]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", conn
)
display(df_2)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.000,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.000,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.000,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.000,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.000,SPY
...,...,...,...,...,...,...,...,...
149,2022-12-13,410.220001,410.489990,399.070007,401.970001,123782500,0.000,SPY
150,2022-12-14,401.609985,405.500000,396.309998,399.399994,108111300,0.000,SPY
151,2022-12-15,394.299988,395.250000,387.890015,389.630005,117705900,0.000,SPY
152,2022-12-16,385.179993,386.579987,381.040009,383.269989,119858000,1.781,SPY


**5. 스크립트의 메인 실행 코드를 작성하세요. 사용자가 주식 심볼(symbol), 시작 날짜(start), 종료 날짜(end)를 입력하여 데이터 수집 및 저장 과정을 시작할 수 있도록 합니다:**

```python
if __name__ == "__main__":
    conn = sqlite3.connect("market_data.sqlite")
    if argv[1] == "bulk":
        symbol = argv[2]
        start_date = argv[3]
        end_date = argv[4]
        save_data_range(symbol, conn, start_date=start_date, end_date=end_date)
        print(f"{symbol} saved between {start_date} and {end_date}")
    elif argv[1] == "last":
        symbol = argv[2]
        calendar = argv[3]
        cal = xcals.get_calendar(calendar)
        today = pd.Timestamp.today().date()
        if cal.is_session(today):
            save_last_trading_session(symbol, conn, today)
            print(f"{symbol} saved")
        else:
            print(f"{today} is not a trading day. Doing nothing.")
    else:
        print("Enter bulk or last")
```

---

**6. 데이터 범위를 저장하려면 터미널에서 다음 명령을 실행합니다:**

```bash
python market_data_sqlite.py bulk SYMBOL START_DATE END_DATE
```

여기서:
- `SYMBOL`은 티커 심볼입니다.
- `START_DATE`는 데이터를 다운로드하려는 첫 날짜입니다.
- `END_DATE`는 데이터를 다운로드하려는 마지막 날짜입니다.

예시:
```bash
python market_data_sqlite.py bulk SPY 2022-01-01 2022-10-20
```

이 명령은 2022년 1월 1일부터 2022년 10월 20일까지 SPY 심볼의 데이터를 다운로드하고 저장합니다.

---

>**중요:**  
>`save_data_range` 함수에서 `to_sql` 메서드의 `if_exists` 인수는 `replace`로 설정되어 있습니다. 이는 함수를 호출할 때마다 테이블이 존재하면 삭제되고 새로 생성됨을 의미합니다. 다양한 주식 데이터를 다른 시간대에 추가하려는 경우 `append`로 설정하는 것이 좋습니다.

---

**마지막 거래일 데이터를 다운로드하려면:**  
```bash
python market_data_sqlite.py last SYMBOL XNYS
```

여기서 `SYMBOL`은 티커 심볼입니다.

>**팁:**  
>`exchange_calendars` 패키지에서 지원되는 캘린더 목록을 가져오려면 다음 명령을 사용하세요:  
>```python
>xcals.get_calendar_names(include_aliases=False)
>```

---

### **작동 방식...**

이전 레시피와 마찬가지로 주식 가격 데이터를 가져오는 동일한 함수를 사용합니다. 이번 레시피에서는 `save_data_range` 함수를 수정하여 연결(`connection`) 인수를 추가하고, 데이터를 CSV로 저장하는 대신 `pandas`의 `to_sql` 메서드를 사용하여 DataFrame 데이터를 SQLite 테이블에 저장합니다. 테이블은 `market_data.sqlite`라는 파일 내부에 있으며, 이는 `connect` 메서드에서 정의됩니다. 이 메서드를 호출하면 Python `sqlite3` 패키지가 파일이 없으면 생성하고, 존재하면 연결합니다.

`save_last_trading_session` 메서드는 연결과 데이터를 다운로드할 날짜를 입력받습니다. 현재 날짜를 시작일과 종료일로 설정해 우리가 만든 `get_stock_data` 함수를 호출합니다. 이 함수는 현재 날짜의 데이터 한 행을 반환합니다. 데이터를 다운로드한 후 SQLite 테이블에 추가됩니다.

`if` 문 아래의 코드는 명령줄에서 호출될 때 실행됩니다. `argv`는 명령줄 인수를 스크립트에 전달하는 데 사용되는 `sys` 모듈에서 제공되는 리스트입니다. 첫 번째 항목(`argv[0]`)은 스크립트 이름 자체를 나타내며, 이후 항목에는 제공된 순서대로 인수가 포함됩니다.  
사용자가 "bulk" 또는 "last"를 입력했는지에 따라 데이터를 범위로 다운로드할지, 마지막 거래일 데이터를 다운로드할지를 결정합니다.  
- "last"가 입력된 경우, `pandas`의 `Timestamp` 클래스를 사용하여 현재 날짜를 확인한 후, `exchange_calendars`를 사용해 해당 날짜가 거래일인지 테스트합니다.  
- 거래일이라면 마지막 거래일 데이터를 추가하고, 아니면 메시지를 출력하며 아무 작업도 수행하지 않습니다.

--- 

### **추가 정보...**

데이터 수집 자동화는 거래 워크플로우에 대해 일관되고 적시적인 입력을 보장합니다. 아래는 생성한 스크립트를 매일 오후 1시(미국 동부 표준시)에 자동으로 실행하도록 설정하는 방법입니다.

---

**Windows**

Windows 사용자는 Python 스크립트를 실행하기 위해 배치 파일(batch file)을 생성할 수 있습니다:

1. **배치 파일 생성:**
   - 새로운 `.bat` 파일을 생성합니다(예: `run_script.bat`).
   - 파일 내부에 다음 내용을 추가합니다:
     ```batch
     @echo off
     CALL conda activate quant-stack
     python path_to_your_script\market_data_sqlite.py %1 %2
     ```

2. **Windows 작업 스케줄러 열기:**
   - `Windows + R` 키를 누르고 `taskschd.msc`를 입력한 후 Enter를 누릅니다.

3. **새 작업 생성:**
   - 작업 스케줄러의 오른쪽 패널에서 "기본 작업 만들기(Create Basic Task)"를 클릭합니다.

4. **작업 이름 및 설명 제공:**
   - **이름(Name):** `Run Market Data Script`
   - **설명(Description):** Python 스크립트를 매주 평일 오후 11시에 실행합니다.

5. **트리거 설정:**
   - **Daily(매일)**을 선택합니다.
   - **시작(Start):** 오늘 날짜와 오후 11시로 설정합니다.
   - **반복 주기(Recur every):** 1일.
   - 고급 설정에서 **주중(Weekdays)**에 체크합니다.

6. **동작(Action) 설정:**
   - **프로그램 시작(Start a program)**을 선택합니다.
   - **프로그램/스크립트(Program/script):** 생성한 `.bat` 파일을 찾아서 선택합니다.
   - **추가 인수(Add arguments):** `last spy xNYS`를 입력합니다.

7. **설정 완료:**
   - **완료(Finish)** 버튼을 클릭합니다.

---

**Mac/Unix/Linux**

Mac 및 Unix 사용자는 Python 스크립트를 실행할 수 있는 실행 가능한 셸 파일을 생성할 수 있습니다:

1. **셸 스크립트 생성:**
   - `run_script.sh`라는 새 파일을 생성합니다.
   - 파일 내부에 다음 내용을 추가합니다:
     ```bash
     #!/bin/bash
     source /path_to_anaconda/anaconda3/bin/activate quant-stack
     python /path_to_your_script/market_data_sqlite.py $1 $2
     ```
   - 실행 권한 부여:
     ```bash
     chmod +x run_script.sh
     ```

2. **크론 테이블 열기:**
   - 터미널을 엽니다.
   - 다음 명령어를 입력합니다:
     ```bash
     crontab -e
     ```

3. **크론 작업 추가:**
   - 스크립트를 평일 오후 11시(미국 동부 표준시)에 실행하려면 다음 줄을 추가합니다:
     ```bash
     0 23 * * 1-5 /path_to_shell_script/run_script.sh last SPY XNYS
     ```

   **참고:**  
   서머타임(Daylight Saving)이나 서버의 시간대에 따라 시간이 조정될 수 있습니다.

4. **저장 및 종료:**
   - (nano 편집기를 사용하는 경우) `Ctrl + O`를 눌러 저장합니다.
   - `Ctrl + X`를 눌러 종료합니다.

5. **크론 작업 확인:**
   - 터미널에서 다음 명령어를 입력하여 작업이 목록에 있는지 확인합니다:
     ```bash
     crontab -l
     ```

---

### **참고 자료...**

SQLite는 매우 빠른 SQL 호환 파일 형식입니다. SQLite를 사용하면 이미 알고 있는 모든 SQL 명령어를 사용할 수 있습니다.

- SQLite 홈페이지: [https://www.sqlite.org/index.html](https://www.sqlite.org/index.html)
- `pandas to_sql` 메서드 문서: [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)
- `exchange_calendars` 패키지 문서: [https://github.com/gerrymanoim/exchange_calendars](https://github.com/gerrymanoim/exchange_calendars)

---

## **PostgreSQL 데이터베이스 서버에 데이터 저장하기**

PostgreSQL(일반적으로 Postgres로 알려짐)은 고급 오픈 소스 관계형 데이터베이스 시스템입니다. 방대한 데이터셋을 처리할 수 있는 능력과 정교한 쿼리 기능을 갖춘 Postgres는 온디스크 옵션보다 향상된 성능이 필요한 알고리즘 트레이더들에게 적합한 선택입니다. Postgres는 또한 AWS와 같은 클라우드 제공업체에서도 널리 사용되는 데이터베이스 옵션입니다.

Postgres의 확장성과 강력함은 특히 고빈도 거래 데이터나 여러 시스템 및 전략이 공유 데이터 자원에 동시에 액세스해야 하는 경우에 중요합니다. 다른 저장 솔루션에 비해 초기 설정이 더 복잡할 수 있지만, 중앙 집중화된 네트워크 액세스와 엄격한 데이터 무결성 검사가 필요한 정교한 거래 운영에서는 이러한 이점이 매력적입니다.

---

### **준비하기...**

이 레시피를 따르려면 원격 Postgres 데이터베이스 서버에 대한 액세스 권한이 있거나 로컬 컴퓨터에 설치된 서버가 필요합니다. Postgres를 로컬 컴퓨터에서 실행하려면 운영 체제에 따라 다음 단계를 따라 설치하세요.

- **Windows:**
  1. Postgres 다운로드 페이지에서 설치 파일을 다운로드합니다.
  2. 설치 파일을 더블 클릭하고 지시에 따라 설치를 진행합니다.
  3. PostgreSQL 폴더의 시작 메뉴에서 pgAdmin을 엽니다.

- **Debian/Ubuntu:**
  다음 명령어를 커맨드 라인에 입력합니다:
  ```bash
  sudo apt-get install libpq-dev python3-dev
  ```

- **Red Hat/CentOS/Fedora:**
  다음 명령어를 커맨드 라인에 입력합니다:
  ```bash
  sudo yum install postgresql-devel python3-devel
  ```

- **macOS(Homebrew 사용):**
  다음 명령어를 터미널에 입력합니다:
  ```bash
  brew install postgresql
  ```

Postgres가 설치되면 운영 체제에 따라 표시된 지침(일반적으로 화면에 출력됨)에 따라 Postgres 데이터베이스 서버를 시작합니다. 서버가 설치된 후, 다음 명령어를 사용해 SQLAlchemy와 psycopg2 드라이버를 설치합니다:

```bash
pip install sqlalchemy psycopg2
```

**자동화 스크립트 작성**  
CRON 작업이나 작업 스케줄러를 사용해 자동으로 실행할 수 있는 스크립트를 작성합니다. 이 레시피에서는 `market_data_postgres.py`라는 Python 스크립트를 생성하고 명령줄에서 실행합니다.

### **어떻게 할까요...**

다음의 모든 코드는 `market_data_postgres.py` 스크립트 파일에 작성되어야 합니다:

1. **필요한 라이브러리 가져오기**:

In [ ]:
# !pip install sqlalchemy psycopg2

In [31]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError

In [32]:
obb.user.preferences.output_type = "dataframe"

데이터베이스 연결 매개변수

In [37]:
username = "admin"
password = "postgre"
host = "127.0.0.1"
port = "5432"
database = "/market_data"

**PostgreSQL서버가 설치되어 있지 않다면, 위 정보를 바탕으로 PostgreSQL 도커 컨테이너를 생성합니다.**

```bash
docker run --name market-data-db \
-e POSTGRES_USER=admin \
-e POSTGRES_PASSWORD=postgre \
-e POSTGRES_DB=market_data \
-p 5432:5432 \
-d postgres
```

In [38]:
# PostgreSQL 데이터베이스 연결 URL 생성
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
# SQLAlchemy 엔진 생성
base_engine = create_engine(DATABASE_URL)

2. **데이터베이스가 존재하지 않을 경우 생성하고 엔진을 반환하는 함수 구현**:

새로운 데이터베이스를 생성하고 해당 데이터베이스의 엔진을 반환하는 함수

In [39]:
def create_database_and_get_engine(db_name, base_engine):
    """
    데이터베이스를 생성하고 해당 데이터베이스에 대한 엔진을 반환하는 함수
    
    Parameters:
        db_name (str): 생성할 데이터베이스 이름
        base_engine: 기본 데이터베이스 연결을 위한 SQLAlchemy 엔진
        
    Returns:
        Engine: 새로 생성된 데이터베이스에 대한 SQLAlchemy 엔진
    """
    # 기본 데이터베이스에 연결
    conn = base_engine.connect()
    # AUTOCOMMIT 모드로 설정
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    try:
        # 데이터베이스 생성 시도
        conn.execute(text(f"CREATE DATABASE {db_name};"))
    except ProgrammingError:
        # 이미 데이터베이스가 존재하는 경우 무시
        pass
    finally:
        # 연결 종료
        conn.close()

    # 새 데이터베이스에 대한 연결 문자열 생성
    conn_str = base_engine.url.set(database=db_name)

    # 새 데이터베이스에 대한 엔진 반환
    return create_engine(conn_str)

데이터베이스를 생성하고 엔진을 가져오기

In [40]:
engine = create_database_and_get_engine("stock_data", base_engine)

3. **이전 두 레시피에서 사용한 `get_stock_data()` 함수 재사용**.

주어진 심볼과 날짜 범위에 대한 주식 데이터를 가져오고 'symbol' 열을 추가하는 함수

In [41]:
def get_stock_data(symbol, start_date=None, end_date=None):
    """
    주어진 심볼과 날짜 범위에 대한 주식 데이터를 가져오는 함수
    
    Parameters:
        symbol (str): 주식 심볼
        start_date (str, optional): 시작 날짜. 기본값은 None
        end_date (str, optional): 종료 날짜. 기본값은 None
        
    Returns:
        DataFrame: 주식 데이터가 포함된 데이터프레임
    """
    # yfinance를 통해 주식 데이터 가져오기
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    # 인덱스를 컬럼으로 변환
    data.reset_index(inplace=True)
    # 심볼 정보 추가
    data["symbol"] = symbol
    return data

4. **`save_data_range` 함수의 `conn` 변수를 `engine`으로 변경**:

가져온 주식 데이터를 PostgreSQL 데이터베이스에 저장하는 함수

In [42]:
def save_data_range(symbol, engine, start_date=None, end_date=None):
    """
    주어진 심볼과 날짜 범위의 주식 데이터를 데이터베이스에 저장하는 함수
    
    Parameters:
        symbol (str): 주식 심볼
        engine: SQLAlchemy 엔진 객체
        start_date (str, optional): 시작 날짜. 기본값은 None
        end_date (str, optional): 종료 날짜. 기본값은 None
    """
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

5. **마지막 거래일의 데이터만 저장하도록 함수 변경**:

마지막 거래 세션의 주식 데이터를 PostgreSQL 데이터베이스에 저장하는 함수

In [43]:
def save_last_trading_session(symbol, engine):
    """
    마지막 거래 세션의 주식 데이터를 데이터베이스에 저장하는 함수
    
    Parameters:
        symbol (str): 주식 심볼
        engine: SQLAlchemy 엔진 객체
    """
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

지정된 날짜 범위에서 여러 주식 심볼에 대한 데이터 저장

In [44]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(
        symbol, engine=engine, start_date="2020-06-01", end_date="2023-01-01"
    )

데이터베이스에서 주식 심볼 "SPY"에 대한 데이터를 읽고 표시

In [45]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", engine)
display(df_1)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.0,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.0,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.0,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.0,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,SPY
...,...,...,...,...,...,...,...,...
648,2022-12-23,379.649994,383.059998,378.029999,382.910004,59857300,0.0,SPY
649,2022-12-27,382.790009,383.149994,379.649994,381.399994,51638200,0.0,SPY
650,2022-12-28,381.329987,383.390015,376.420013,376.660004,70911500,0.0,SPY
651,2022-12-29,379.630005,384.350006,379.079987,383.440002,66970900,0.0,SPY


거래량이 100,000,000 이상인 "SPY" 데이터를 읽고 표시

In [46]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", engine
)
display(df_2)

,date,open,high,low,close,volume,dividend,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.000,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.000,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.000,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.000,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.000,SPY
...,...,...,...,...,...,...,...,...
149,2022-12-13,410.220001,410.489990,399.070007,401.970001,123782500,0.000,SPY
150,2022-12-14,401.609985,405.500000,396.309998,399.399994,108111300,0.000,SPY
151,2022-12-15,394.299988,395.250000,387.890015,389.630005,117705900,0.000,SPY
152,2022-12-16,385.179993,386.579987,381.040009,383.269989,119858000,1.781,SPY


데이터베이스를 구축하는 가장 좋은 방법은 먼저 과거 데이터를 다운로드하는 것입니다. 이것은 한 번만 수행하여 과거 데이터를 "백필(backfill)"하고, 그 다음에는 매일 장 마감 후에 스크립트가 실행되도록 스케줄링하여 해당 날짜의 데이터를 계속 수집합니다.

Mac과 Windows에서 Python 작업을 스케줄링하는 방법에 대한 많은 참고자료가 있으므로 여기서는 자세히 다루지 않겠습니다.

다음 두 가지 방법이 잘 작동합니다:

Mac/Linux에서 스케줄링: https://theautomatic.net/2020/11/18/how-to-schedule-a-python-script-on-a-mac/
Windows에서 스케줄링: https://www.jcchouinard.com/python-automation-using-task-scheduler/

**6. 스크립트의 메인 실행 코드를 작성합니다.**  
이 코드는 데이터베이스 연결을 생성하고, Python 코드를 호출하여 데이터를 다운로드하고 저장합니다.

```python
if __name__ == "__main__":
    username = ""
    password = ""
    host = "127.0.0.1"
    port = "5432"
    database = "market_data"
    
    DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
    base_engine = create_engine(DATABASE_URL)
    engine = create_database_and_get_engine("stock_data", base_engine)
    
    if argv[1] == "bulk":
        symbol = argv[2]
        start_date = argv[3]
        end_date = argv[4]
        save_data_range(symbol, engine, start_date=start_date, end_date=end_date)
        print(f"{symbol} saved between {start_date} and {end_date}")
    elif argv[1] == "last":
        symbol = argv[2]
        calendar = argv[3]
        cal = xcals.get_calendar(calendar)
        today = pd.Timestamp.today().date()
        if cal.is_session(today):
            save_last_trading_session(symbol, engine, today)
            print(f"{symbol} saved")
        else:
            print(f"{today} is not a trading day. Doing nothing.")
```

---

**7. 데이터 범위를 저장하려면 터미널에서 다음 명령어를 실행합니다:**

```bash
python market_data_sqlite.py bulk SYMBOL START_DATE END_DATE
```

여기서:  
- `SYMBOL`은 티커 심볼입니다.  
- `START_DATE`는 데이터를 다운로드하려는 첫 번째 날짜입니다.  
- `END_DATE`는 데이터를 다운로드하려는 마지막 날짜입니다.  

**예시:**  
```bash
python market_data_sqlite.py bulk SPY 2022-01-01 2022-10-20
```

이 명령어는 SPY 심볼의 데이터를 2022-01-01부터 2022-10-20까지 다운로드하고 저장합니다.

---

>**중요:**  
>Postgres는 기본적으로 사용자 이름과 비밀번호를 설정하지 않습니다. 운영 체제 및 설치된 도구(예: pgAdmin)에 따라 사용자 이름과 비밀번호를 설정하는 과정이 다릅니다. 데이터 무결성을 보장하려면 이 두 가지를 반드시 설정하세요. Python 코드에서 사용하는 자격 증명을 `.env` 파일에 저장하고, `dotenv` 패키지를 사용하여 환경 변수에서 설정하는 것이 좋습니다.

---

### **작동 방식...**

1. 필요한 Python 라이브러리를 가져옵니다.  
이 레시피에서는 SQLAlchemy를 도입하여 데이터베이스에 연결하고 상호 작용할 수 있는 도구를 제공합니다.

2. 새 데이터베이스를 생성하는 함수를 구현합니다.  
데이터베이스가 이미 존재하면 단순히 연결합니다. `AUTOCOMMIT` 격리 수준은 PostgreSQL의 트랜잭션 블록 내에서 데이터베이스를 생성하는 제한을 우회하도록 설정됩니다. 데이터베이스 생성 후, 함수는 엔진을 반환합니다.

3. 금융 시장 데이터를 가져오고, `pandas to_sql` 메서드를 사용하여 데이터를 Postgres 데이터베이스에 저장하는 일련의 함수를 작성합니다.  
이전 레시피와 유사하게 새 티커 데이터의 대량 저장과 마지막 거래일 데이터를 저장하기 위한 두 가지 함수를 만듭니다.

4. 메인 코드 실행 블록에서는 연결 매개변수를 설정하고 "엔진"을 생성합니다.  
이는 Pandas를 통해 Postgres 데이터베이스에 연결하는 방법입니다. 나머지 코드는 이전 레시피와 동일하게 작동합니다.

---

### **추가 정보...**

SQLAlchemy는 Python에서 데이터베이스와 상호 작용하기 위한 강력한 도구 키트로, Python 애플리케이션과 관계형 데이터베이스 간의 원활한 통신을 가능하게 합니다.  
SQLAlchemy의 **객체 관계 매핑(Object Relational Mapping, ORM)** 계층을 통해 개발자는 네이티브 Python 클래스를 사용하여 데이터베이스와 상호 작용할 수 있으며, 로우 SQL의 복잡성을 추상화합니다.  
또한 데이터베이스에 구애받지 않는 특성(database-agnostic)을 가지고 있어, 애플리케이션을 한 번 구축하면 다양한 데이터베이스 백엔드에 최소한의 변경으로 배포할 수 있어 유연성과 확장성을 보장합니다.  
이는 로컬 컴퓨터에서 개발용 데이터베이스를 구축하고 이후 원격 서버로 전환할 때 매우 유용합니다.

---

### **참고 자료...**

SQLAlchemy의 고급 기능을 더 탐구하고 싶은 사람들을 위해, Python 클래스를 사용해 데이터베이스를 모델링하는 방법에 대한 빠른 시작 가이드가 SQLAlchemy 웹사이트에 제공됩니다:  
[SQLAlchemy ORM Quick Start Guide](https://docs.sqlalchemy.org/en/20/orm/quickstart.html)

- [SQLAlchemy 문서 페이지](https://docs.sqlalchemy.org/en/20/index.html)  
- [PostgreSQL 홈페이지](https://www.postgresql.org)

Postgres 데이터베이스 서버를 관리하는 좋은 방법은 무료 소프트웨어인 **pgAdmin**을 사용하는 것입니다.
pgAdmin은 서버 리소스를 관리하고 쿼리를 작성할 수 있는 그래픽 사용자 인터페이스를 제공합니다. pgAdmin에 대해 자세히 알아보고 다운로드하려면 다음 URL을 방문하세요: https://www.pgadmin.org/.

---

## **초고속 HDF5 형식으로 데이터 저장하기**

**계층적 데이터 형식(HDF)** 는 파일 형식 모음으로, HDF4와 HDF5로 구성되어 있으며 대량 데이터를 계층적으로 저장하고 관리하기 위해 설계되었습니다. HDF5는 미국 국립 슈퍼컴퓨터 애플리케이션 센터에서 처음 개발된 오픈 소스 형식으로, 대규모 복합 이질적 데이터셋을 수용합니다. 디렉토리와 같은 구조를 사용하여 파일 내 데이터를 유연하게 조직할 수 있으며, 이는 컴퓨터의 파일 관리 방식과 유사합니다.  

HDF5는 두 가지 주요 객체 유형을 포함합니다:
- **데이터셋:** 형식화된 다차원 배열.
- **그룹:** 데이터셋 및 다른 그룹을 포함할 수 있는 컨테이너 구조.

Python에서는 HDF5를 지원하는 두 개의 라이브러리가 있습니다:
1. **h5py:** HDF5 구조에 대한 고급 및 저급 접근 모두를 제공합니다.
2. **PyTables:** 고급 인터페이스와 고급 인덱싱 및 쿼리 기능을 제공하는 라이브러리입니다.

**PyTables**는 대규모 데이터셋 및 계층적 데이터베이스를 관리하기 위해 설계된 Python 라이브러리로, HDF5 파일 형식을 사용하여 데이터를 효율적으로 저장, 접근, 처리할 수 있는 도구를 제공합니다. 이는 고성능 및 데이터 집중형 애플리케이션에 적합합니다.

HDF5를 사용하여 데이터를 저장하면 계층 구조로 관련 데이터를 저장할 때 유리합니다. 예를 들어, 주식의 기초 데이터, 선물 만기 데이터, 옵션 체인 데이터를 저장하는 데 유용합니다.

---

### **준비하기...**

이 레시피를 따르기 위해서는 HDF5 형식의 데이터에 접근하기 위해 pandas가 사용하는 **PyTables**가 컴퓨터에 설치되어 있어야 합니다.  
다음 conda 명령어를 사용하여 PyTables를 설치할 수 있습니다:

In [ ]:
!conda install -c conda-forge pytables -y

---

### **어떻게 할까요...**

이 레시피에서는 Jupyter 노트북으로 돌아갑니다:

1. **필요한 라이브러리를 가져오고 변수를 설정합니다:**

In [1]:
import warnings

In [2]:
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb

In [3]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

In [4]:
STOCKS_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"

In [5]:
ticker = "SPY"
root = "ES"

2. **OpenBB 플랫폼을 사용하여 SPY 주식 가격 및 옵션 체인 데이터를 로드합니다:**

Fetches historical price data for the equity "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_equity'

In [6]:
spy_equity = obb.equity.price.historical(
    ticker, start_date="2021-01-01", provider="yfinance"
)

Fetches options chains for the equity "SPY" using the "cboe" provider and stores it in 'spy_chains'

In [7]:
spy_chains = obb.derivatives.options.chains(ticker, provider="cboe")

Gets the unique expiration dates from 'spy_chains'

In [8]:
spy_expirations = spy_chains.expiration.astype(str).unique().tolist()

Fetches historical price data for a specific option of "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_historic'

In [9]:
spy_historic = obb.equity.price.historical(
    ticker + spy_expirations[-10].replace("-", "")[2:] + "C" + "00400000",
    start_date="2021-01-01",
    provider="yfinance",
)

**3. 데이터를 HDF5 파일에 저장:**

Save stock data to HDF5 store

In [10]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    store.put("equities/spy/stock_prices", spy_equity)
    store.put("equities/spy/options_prices", spy_historic)
    store.put("equities/spy/chains", spy_chains)

**4. HDF5 파일에서 데이터를 pandas DataFrame으로 읽기:**

Load stock data from HDF5 store

In [ ]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    spy_prices = store["equities/spy/stock_prices"]
    spy_options = store["equities/spy/options_prices"]
    spy_chains = store["equities/spy/chains"]

**5. e-mini 선물 만기를 반복하면서 각 만기의 데이터를 별도의 파일 경로에 저장:**

Save futures data to HDF5 store

In [11]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    for i in range(24, 31):
        expiry = f"20{i}-12"
        df = obb.derivatives.futures.historical(
            symbol=[root],
            expiry=expiry,
            start_date="2021-01-01",
        )
        df.rename(columns={"close": expiry}, inplace=True)
        prices = df[expiry]

        store.put(f"futures/{root}/{expiry}", prices)

**6. ETF와 동일한 방식으로 데이터 읽기:**

Load futures data from HDF5 store

In [13]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    es_prices = store[f"futures/{root}/2024-12"]

In [14]:
display(spy_prices)
display(spy_options)
display(es_prices)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,110210800,0.0,0.0,0.0
2021-01-05,368.100006,372.500000,368.049988,371.329987,66426200,0.0,0.0,0.0
2021-01-06,369.709991,376.980011,369.119995,373.549988,107997700,0.0,0.0,0.0
2021-01-07,376.100006,379.899994,375.910004,379.100006,68766800,0.0,0.0,0.0
2021-01-08,380.589996,381.489990,377.100006,381.260010,71677200,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-06-12,541.630005,544.119995,540.299988,541.359985,63251300,0.0,0.0,0.0
2024-06-13,543.150024,543.330017,539.590027,542.450012,44760900,0.0,0.0,0.0
2024-06-14,540.880005,542.809998,539.849976,542.780029,40089900,0.0,0.0,0.0


,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2022-09-13,64.449997,64.449997,64.449997,64.449997,23,0.0,0.0
2022-09-15,61.419998,61.419998,61.000000,61.000000,5,0.0,0.0
2022-09-16,60.830002,60.830002,58.000000,58.000000,2,0.0,0.0
2022-09-19,57.669998,60.000000,57.570000,60.000000,411,0.0,0.0
2022-09-20,60.000000,60.000000,56.000000,56.099998,124,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-07,147.220001,147.220001,147.220001,147.220001,1,0.0,0.0
2024-06-12,154.710007,154.710007,154.009995,154.009995,131,0.0,0.0
2024-06-13,154.000000,154.000000,151.520004,153.130005,16,0.0,0.0


date
2000-09-18    1467.5
2000-09-19    1478.5
2000-09-20    1469.5
2000-09-21    1469.5
2000-09-22    1468.5
               ...  
2024-06-13    5438.5
2024-06-14    5437.5
2024-06-17    5478.5
2024-06-18    5491.0
2024-06-19    5493.0
Name: 2024-12, Length: 6000, dtype: float64

---

### **작동 방식...**

먼저 OpenBB 플랫폼을 사용해 SPY ETF의 과거 가격 데이터와 옵션 체인 데이터를 다운로드합니다. 그런 다음, 만기일 데이터를 추출하고 `obb.equity.price.historical` 메서드를 사용해 옵션 티커 심볼을 생성하여 과거 데이터를 요청합니다. 이 데이터를 pandas DataFrame에 저장한 후, `pandas HDFStore` 메서드를 사용해 HDF5 파일(`assets.h5`)을 엽니다. Python의 `with` 문은 컨텍스트 관리자 제어 하에 일련의 명령문을 실행할 수 있는 환경을 제공합니다. 여기서 HDF5 파일은 pandas HDFStore 객체로 열리며, `put` 메서드를 사용해 DataFrame 데이터를 HDF5 파일에 손쉽게 저장합니다.

선물 예제는 데이터 소스 목록을 반복하여 각 데이터를 HDF5 파일 내 별도의 경로로 저장하는 방법을 보여줍니다. 먼저 HDF5 파일을 열고, 만기일 목록을 반복하며, 각 만기일에 대해 OpenBB 플랫폼을 사용해 가격 데이터를 다운로드하고, 열 이름을 변경한 뒤, 데이터를 HDF5 파일에 저장합니다.

---

### **추가 정보...**

HDF5는 숫자 데이터 저장을 위한 가장 빠른 온디스크 열형 데이터 저장 형식 중 하나로 간주됩니다. 이 형식은 압축된 CSV 형식과 유사한 작은 메모리 사용량을 공유합니다. 또 다른 파일 형식은 Parquet으로, 효율적인 데이터 압축 및 인코딩을 제공하는 이진 열형 저장 형식입니다. Parquet은 PyArrow 라이브러리를 통해 pandas에서 사용할 수 있습니다.

---

### **참고 자료...**
- pandas HDFStore 문서: [https://pandas.pydata.org/docs/reference/api/pandas.HDFStore.put.html](https://pandas.pydata.org/docs/reference/api/pandas.HDFStore.put.html)
- PyTables 문서: [https://www.pytables.org](https://www.pytables.org)
- PyArrow의 Python 바인딩 문서: [https://arrow.apache.org/docs/python/index.html](https://arrow.apache.org/docs/python/index.html)

---